# Parse backtest result

In [1]:
import json, pandas as pd


file_name = '../backtest_results/backtest-20220211-155505.json'

with open(file_name) as f:
    json_file = json.load(f)

df = pd.json_normalize(json_file['balance']).fillna(0.0) #pd.DataFrame.from_dict(json_file['data'], orient='index')
#df.set_index('date',inplace=True)
#df
dx = df.set_index('date').diff()
dx
#df

,USDT,LUNA,ICP,AAVE,FTM,CHZ
date,,,,,,
2021-12-29 06:51:00,NaN,NaN,NaN,NaN,NaN,NaN
2021-12-29 08:31:00,-0.450809,-1.74,6.06393,0.000000,0.000,0.000
2021-12-29 10:11:00,-0.056167,0.00,-6.06000,0.595404,0.000,0.000
2021-12-29 11:51:00,0.000000,0.00,0.00000,0.000000,0.000,0.000
2021-12-29 13:31:00,0.012736,0.00,0.00000,-0.595000,65.934,0.000
2021-12-29 15:11:00,0.000000,0.00,0.00000,0.000000,0.000,0.000
2021-12-29 16:51:00,0.000000,0.00,0.00000,0.000000,0.000,0.000
2021-12-29 18:31:00,0.000000,0.00,0.00000,0.000000,0.000,0.000
2021-12-29 20:11:00,0.000000,0.00,0.00000,0.000000,0.000,0.000
